In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import flopy
import flopy.utils.binaryfile as bf
from mfsetup import MF6model
from mfsetup.discretization import cellids_to_kij
from gisutils import df2shp
import mfexport

flopy is installed in /Users/aleaf/Documents/GitHub/flopy/flopy


In [ ]:
m = MF6model.setup_from_yaml('pleasant_lgr_parent.yml')

loading configuration file pleasant_lgr_parent.yml...

Setting up plsnt_lgr_parent model from data in None


arguments to MFSimulation:
sim_name: pleasant_lgr
version: mf6
exe_name: mf6
sim_ws: /Users/aleaf/Documents/GitHub/modflow-setup/examples/pleasant_lgr

other arguments:
options: {}



arguments to ModflowGwf:
modelname: plsnt_lgr_parent
version: mf6
simulation: sim_name = pleasant_lgr
sim_path = /Users/aleaf/Documents/GitHub/modflow-setup/examples/pleasant_lgr
exe_name = mf6

###################
Package mfsim.nam
###################

package_name = mfsim.nam
filename = mfsim.nam
package_type = nam
model_or_simulation_package = simulation
simulation_name = pleasant_lgr



list: /Users/aleaf/Documents/GitHub/modflow-setup/examples/pleasant_lgr/plsnt_lgr_parent.list
print_input: True
print_flows: False
save_flows: True
newtonoptions: ['under_relaxation']

other arguments:
list_filename_fmt: {}.list
options: {'newton': True,
 'newton_under_relaxation': True,
 'newtonoptions': ['unde

In [ ]:
m

In [ ]:
m.cfg.keys()

In [ ]:
m.cfg['dis']

In [ ]:
m.inset

In [ ]:
inset = m.inset['plsnt_lgr_inset']

l, r, b, t = m.modelgrid.extent
layer = 0

fig, ax = plt.subplots(figsize=(10, 10))
parent_mv = flopy.plot.PlotMapView(model=m, ax=ax, layer=layer)
inset_mv = flopy.plot.PlotMapView(model=inset, ax=ax, layer=layer)

vconn = inset.lak.connectiondata.array[inset.lak.connectiondata.array['claktype'] == 'vertical']
k, i, j = cellids_to_kij(vconn['cellid'])
lakeconnections = np.zeros((inset.nrow, inset.ncol))
lakeconnections[i, j] = np.array(k)
lakeconnections = np.ma.masked_array(lakeconnections, mask=lakeconnections == 0)
qmi = inset_mv.plot_array(lakeconnections)

#inset_mv.plot_bc('LAK', color='navy')
#parent_mv.plot_bc('WEL_0', color='red')

lcp = parent_mv.plot_grid(lw=0.5, ax=ax)
lci = inset_mv.plot_grid(lw=0.5)
ax.set_ylim(b, t)
ax.set_xlim(l, r)
ax.set_aspect(1)
plt.colorbar(qmi)

In [ ]:
m.write_input()

In [ ]:
# write the SFR package again using SFRmaker,
# because flopy doesn't check the packagedata against the idomain before writing
for model in m, m.inset['plsnt_lgr_inset']:
    if hasattr(model, 'sfr'):
        sfr_package_filename = os.path.join(model.model_ws, model.sfr.filename)
        model.sfrdata.write_package(sfr_package_filename,
                                    idomain=model.dis.idomain.array,
                                    version='mf6',
                                    options=['save_flows',
                                             'BUDGET FILEOUT {}.sfr.cbc'.format(model.name),
                                             'STAGE FILEOUT {}.sfr.stage.bin'.format(model.name),
                                             'mover'
                                               ]
                                    )

In [ ]:
m.simulation.run_simulation()

In [ ]:
os.getcwd()

In [ ]:
tmr_parent_headsobj = bf.HeadFile('../data/pleasant/pleasant.hds')
lgr_parent_headsobj = bf.HeadFile('plsnt_lgr_parent.hds')
lgr_inset_headsobj = bf.HeadFile('plsnt_lgr_inset.hds')

tmr_parent_hds = tmr_parent_headsobj.get_data(kstpkper=(4, 12))
lgr_parent_hds = lgr_parent_headsobj.get_data(kstpkper=(0, 12))
lgr_inset_hds = lgr_inset_headsobj.get_data(kstpkper=(0, 12))

lgr_parent_hds = np.ma.masked_array(lgr_parent_hds, mask=lgr_parent_hds == 1e30)
lgr_inset_hds = np.ma.masked_array(lgr_inset_hds, mask=lgr_inset_hds == 1e30)

In [ ]:
layer = 0

fig, ax = plt.subplots(figsize=(10, 10))
parent_mv = flopy.plot.PlotMapView(model=m, ax=ax, layer=layer)
inset_mv = flopy.plot.PlotMapView(model=inset, ax=ax, layer=layer)

pctr = parent_mv.contour_array(lgr_parent_hds, levels=np.arange(290, 315))
ictr = inset_mv.contour_array(lgr_inset_hds, levels=np.arange(290, 315))
#qmp = parent_mv.plot_array(lgr_parent_hds)
#qmi = inset_mv.plot_array(lgr_inset_hds)

lcp = parent_mv.plot_grid(lw=0.5, ax=ax)
lci = inset_mv.plot_grid(lw=0.5)
ax.set_ylim(b, t)
ax.set_xlim(l, r)
ax.set_aspect(1)
#plt.colorbar(qmp)

In [ ]:
for model in m, inset:
    mfexport.export(model, model.modelgrid, output_path='postproc/{}/'.format(model.name))

In [ ]:
for model in m, inset:
    mfexport.summarize(model, output_path='postproc/{}/'.format(model.name))

In [ ]:
outpath='postproc/{}/shps'.format(inset.name)
if not os.path.isdir(outpath):
    os.makedirs(outpath)
connectiondata = pd.DataFrame(inset.lak.connectiondata.array)
k, i, j = cellids_to_kij(connectiondata['cellid'])
connectiondata['k'] = k
connectiondata['i'] = i
connectiondata['j'] = j
#connectiondata.drop('cellid', axis=1, inplace=True)
polygons = np.reshape(inset.modelgrid.polygons, (inset.modelgrid.nrow, inset.modelgrid.ncol))
connectiondata['geometry'] = polygons[i, j]
df2shp(connectiondata, os.path.join(outpath, 'lake_connections.shp'), epsg=3070)